In [ ]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog, QMessageBox, QComboBox
import mysql.connector
from mysql.connector import Error
from PIL import Image

from deepface import DeepFace
import cv2
from PyQt5.QtWidgets import QMessageBox
from deepface import DeepFace
import traceback
import face_recognition
import numpy as np
from PyQt5.QtWidgets import QMessageBox
import io
from PIL import Image
import mysql.connector
from mysql.connector import Error
from io import BytesIO


# MySQL configuration
mysql_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "images_db"
}

# Function to read and convert an image into byte array
def read_image(image_path):
    with Image.open(image_path) as img:
        img_byte_array = io.BytesIO()
        img.save(img_byte_array, format=img.format)
        return img_byte_array.getvalue()

# Insert image into the database
def insert_image_to_db(cursor, image_name, image_data):
    cursor.execute(
        "INSERT INTO images_store (image_name, image_column) VALUES (%s, %s)",
        (image_name, image_data),
    )

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.setMinimumSize(QtCore.QSize(1200, 633))
        MainWindow.setMaximumSize(QtCore.QSize(1200, 633))
        MainWindow.setBaseSize(QtCore.QSize(1200, 633))

        # Central widget setup
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")

        # Add tab widget
        self.tabWidget = QtWidgets.QTabWidget(self.centralwidget)
        self.tabWidget.setGeometry(QtCore.QRect(0, 0, 1200, 600))

        # First Tab: Face Recognition from Database
        self.tab1 = QtWidgets.QWidget()
        self.tab1.setObjectName("tab1")

        self.image_viewer = QtWidgets.QLabel(self.tab1)
        self.image_viewer.setGeometry(QtCore.QRect(100, 50, 1000, 351))
        self.image_viewer.setScaledContents(True)
        self.image_viewer.setObjectName("image_viewer")

        self.load_button = QtWidgets.QPushButton(self.tab1)
        self.load_button.setGeometry(QtCore.QRect(300, 450, 221, 41))
        self.load_button.setObjectName("load_button")

        self.match_button = QtWidgets.QPushButton(self.tab1)
        self.match_button.setGeometry(QtCore.QRect(600, 450, 221, 41))
        self.match_button.setObjectName("match_button")

        # ComboBox for model selection
        self.model_combobox = QComboBox(self.tab1)
        self.model_combobox.setGeometry(QtCore.QRect(450, 500, 221, 41))
        self.model_combobox.setObjectName("model_combobox")
        self.model_combobox.addItems(["VGG-Face", "OpenFace", "Facenet", "DeepFace"])

        self.tabWidget.addTab(self.tab1, "Face Recognition")

        # Second Tab: Real-Time Face Recognition (Webcam)
        self.tab2 = QtWidgets.QWidget()
        self.tab2.setObjectName("tab2")

        # Webcam-related widgets in second tab
        self.webcam_image_viewer = QtWidgets.QLabel(self.tab2)
        self.webcam_image_viewer.setGeometry(QtCore.QRect(100, 50, 1000, 351))
        self.webcam_image_viewer.setScaledContents(True)
        self.webcam_image_viewer.setObjectName("webcam_image_viewer")

        self.open_webcam_button = QtWidgets.QPushButton(self.tab2)
        self.open_webcam_button.setGeometry(QtCore.QRect(50, 450, 221, 41))
        self.open_webcam_button.setObjectName("open_webcam_button")

        self.stop_webcam_button = QtWidgets.QPushButton(self.tab2)
        self.stop_webcam_button.setGeometry(QtCore.QRect(450, 450, 221, 41))
        self.stop_webcam_button.setObjectName("stop_webcam_button")

        self.model_combobox_2 = QComboBox(self.tab2)
        self.model_combobox_2.setGeometry(QtCore.QRect(450, 500, 221, 41))
        self.model_combobox_2.setObjectName("model_combobox_2")
        self.model_combobox_2.addItems(["VGG-Face", "OpenFace", "Facenet", "DeepFace"])
        # Add this button to your UI in the second tab
        self.capture_button = QtWidgets.QPushButton(self.tab2)
        self.capture_button.setGeometry(QtCore.QRect(900, 450, 221, 41))
        self.capture_button.setObjectName("capture_button")
        self.capture_button.setText("Capture")

        self.tabWidget.addTab(self.tab2, "Real-Time Face Recognition")
        # Third Tab: Manage Dataset
        self.tab3 = QtWidgets.QWidget()
        self.tab3.setObjectName("tab3")
        self.name_edit_box = QtWidgets.QLineEdit(self.tab3)
        self.name_edit_box.setGeometry(QtCore.QRect(20, 60, 200, 30))
        self.name_edit_box.setObjectName("name_edit_box")
        self.add_face_button = QtWidgets.QPushButton(self.tab3)
        self.add_face_button.setGeometry(QtCore.QRect(20, 120, 200, 40))
        self.add_face_button.setObjectName("add_face_button")
        self.clear_button = QtWidgets.QPushButton(self.tab3)
        self.clear_button.setGeometry(QtCore.QRect(20, 180, 200, 40))
        self.clear_button.setObjectName("clear_button")
        self.remove_face_button = QtWidgets.QPushButton(self.tab3)
        self.remove_face_button.setGeometry(QtCore.QRect(20, 240, 200, 40))
        self.remove_face_button.setObjectName("remove_face_button")
        self.face_list_button = QtWidgets.QPushButton(self.tab3)
        self.face_list_button.setGeometry(QtCore.QRect(20, 300, 200, 40))
        self.face_list_button.setObjectName("face_list_button")
        self.img_label = QtWidgets.QLabel(self.tab3)
        self.img_label.setGeometry(QtCore.QRect(300, 50, 400, 360))
        self.img_label.setScaledContents(True)
        self.img_label.setObjectName("img_label")
        self.list_label = QtWidgets.QLabel(self.tab3)
        self.list_label.setGeometry(QtCore.QRect(750, 50, 400, 360))
        self.list_label.setScaledContents(False)
        self.list_label.setWordWrap(True)
        self.list_label.setObjectName("list_label")
        self.tabWidget.addTab(self.tab3, "Manage Dataset")
        # Create the "Clear Label" button on the third tab
        self.clear_label_button = QtWidgets.QPushButton(self.tab3)
        self.clear_label_button.setGeometry(QtCore.QRect(20, 360, 200, 40))  # Adjust position as needed
        self.clear_label_button.setObjectName("clear_label_button")
        self.clear_label_button.setText("Clear Label")

        # Connect the button to the method that clears the label
        self.clear_label_button.clicked.connect(self.clear_label)

        # Connect buttons to functions
        self.load_button.clicked.connect(self.load_image_for_recognition)
        self.match_button.clicked.connect(self.match_face)
        self.open_webcam_button.clicked.connect(self.start_webcam)
        self.stop_webcam_button.clicked.connect(self.stop_webcam)
        # Button connections
        self.add_face_button.clicked.connect(self.add_face_to_dataset)
        self.clear_button.clicked.connect(self.clear_face_entry)
        self.remove_face_button.clicked.connect(self.remove_face_from_dataset)
        self.face_list_button.clicked.connect(self.list_faces_in_dataset)
        self.capture_button.clicked.connect(self.capture_frame_and_stop_webcam)

        # Central widget setup
        MainWindow.setCentralWidget(self.centralwidget)

        # Menubar and statusbar setup
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 801, 26))
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        MainWindow.setStatusBar(self.statusbar)

        # Call retranslateUi to set up text for all UI components
        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        
        MainWindow.setWindowTitle(_translate("MainWindow", "Face Recognition and Management"))
        self.capture_button.setText(_translate("MainWindow", "Capture"))
        self.load_button.setText(_translate("MainWindow", "Load Image"))
        self.match_button.setText(_translate("MainWindow", "Match"))
        self.open_webcam_button.setText(_translate("MainWindow", "Open Webcam"))
        self.stop_webcam_button.setText(_translate("MainWindow", "Stop Webcam"))
        self.add_face_button.setText(_translate("MainWindow", "Add Face"))
        self.clear_button.setText(_translate("MainWindow", "Clear"))
        self.remove_face_button.setText(_translate("MainWindow", "Remove Face"))
        self.face_list_button.setText(_translate("MainWindow", "Face List"))

    def load_image_for_recognition(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(None, "Choose Image", "", "Image Files (*.jpg *.jpeg *.png)", options=options)
        if file_path:
            self.recognition_image_path = file_path
            pixmap = QtGui.QPixmap(file_path)
            self.image_viewer.setPixmap(pixmap)
    def clear_label(self):
        # Clears the text of the QLabel used for the list
        self.list_label.setText("")  # This clears the text in the label

    def match_face(self):
        if not hasattr(self, 'recognition_image_path'):
            QMessageBox.warning(None, "Error", "Please load an image to match.")
            return

        # Get selected model from ComboBox
        selected_model = self.model_combobox.currentText()

        # Perform recognition using DeepFace
        self.recognize_face_with_deepface(self.recognition_image_path, selected_model)

    def start_webcam(self):
        # Initialize the webcam (OpenCV)
        self.cap = cv2.VideoCapture(0)
    
        # Check if the webcam opened successfully
        if not self.cap.isOpened():
            QMessageBox.warning(None, "Error", "Could not open webcam.")
            return
    
        self.webcam_running = True
        self.capture_frame()

    def stop_webcam(self):
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.cap.release()
        self.webcam_running = False
        self.webcam_image_viewer.clear()

    def capture_frame(self):
        if self.webcam_running:
            ret, frame = self.cap.read()
            if ret:
                # Perform face detection and draw bounding boxes
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_locations = face_recognition.face_locations(rgb_frame)
    
                for (top, right, bottom, left) in face_locations:
                    # Draw bounding box around detected face
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
    
                # Convert the frame to QPixmap for displaying in QLabel
                height, width, channel = frame.shape
                bytes_per_line = 3 * width
                q_img = QtGui.QImage(frame.data, width, height, bytes_per_line, QtGui.QImage.Format_BGR888)
                self.webcam_image_viewer.setPixmap(QtGui.QPixmap.fromImage(q_img))
    
            # Call capture_frame again to get the next frame
            QtCore.QTimer.singleShot(1, self.capture_frame)

    def capture_face(self):
        self.identify_faces = True

    def load_encodings_from_database(self):
        known_face_encodings = []
        known_face_names = []
        try:
            db_connection = mysql.connector.connect(
                host="localhost",
                user="root",
                password="",  
                database="images_db" 
            )
            cursor = db_connection.cursor()
            cursor.execute("SELECT image_name, image_column FROM images_store")  
            results = cursor.fetchall()

            for name, image_blob in results:
                image = face_recognition.load_image_file(BytesIO(image_blob))  
                face_encodings = face_recognition.face_encodings(image)

                if face_encodings:  
                    known_face_encodings.append(face_encodings[0])  
                    known_face_names.append(name) 

            cursor.close()
            db_connection.close()

            return known_face_encodings, known_face_names

        except mysql.connector.Error as err:
            print(f"Error: {err}")
            return [], [] 
    def recognize_face_with_deepface(self, image_path, model_name):
        try:
            # Load known encodings and names from the database
            known_face_encodings, known_face_names = self.load_encodings_from_database()
    
            if not known_face_encodings:
                QMessageBox.warning(None, "No Known Faces", "No faces found in the database for recognition.")
                return
    
            # Load the image for recognition
            image = cv2.imread(image_path)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
            face_locations = face_recognition.face_locations(rgb_image)
            face_encodings = face_recognition.face_encodings(rgb_image, face_locations)
    
            if not face_encodings:
                QMessageBox.warning(None, "No Faces Detected", "No faces were detected in the uploaded image.")
                return
    
            # Process each detected face
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
    
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
    
                # Draw a bounding box around the face
                cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)  # Red color with thickness of 2
    
                # Put the name below the bounding box
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(image, name, (left, bottom + 20), font, 1.0, (255, 255, 255), 1)
    
            # Convert the image back to Qt format for displaying
            height, width, channel = image.shape
            bytes_per_line = 3 * width
            q_img = QtGui.QImage(image.data, width, height, bytes_per_line, QtGui.QImage.Format_BGR888)
    
            # Show the image with bounding box and name
            self.image_viewer.setPixmap(QtGui.QPixmap.fromImage(q_img))
    
        except Exception as e:
            error_message = f"An error occurred while processing the image:\n\n{str(e)}"
            QMessageBox.warning(None, "Error", error_message)
    # Functions for managing dataset
    def add_face_to_dataset(self):
        name = self.name_edit_box.text()
        if not name:
            QMessageBox.warning(None, "Error", "Enter a name.")
            return
        file_path, _ = QFileDialog.getOpenFileName(None, "Choose Image", "", "Images (*.jpg *.jpeg *.png)")
        if file_path:
            try:
                image_data = read_image(file_path)
                db_connection = mysql.connector.connect(**mysql_config)
                cursor = db_connection.cursor()
                insert_image_to_db(cursor, name, image_data)
                db_connection.commit()
                cursor.close()
                db_connection.close()
                self.img_label.setPixmap(QtGui.QPixmap(file_path))
                QMessageBox.information(None, "Success", "Face added successfully.")
            except Error as e:
                QMessageBox.critical(None, "Error", str(e))

    def clear_face_entry(self):
        self.name_edit_box.clear()
        self.img_label.clear()

    def remove_face_from_dataset(self):
        name = self.name_edit_box.text()
        if not name:
            QMessageBox.warning(None, "Error", "Enter a name.")
            return
        try:
            db_connection = mysql.connector.connect(**mysql_config)
            cursor = db_connection.cursor()
            cursor.execute("DELETE FROM images_store WHERE image_name = %s", (name,))
            db_connection.commit()
            cursor.close()
            db_connection.close()
            QMessageBox.information(None, "Success", "Face removed successfully.")
        except Error as e:
            QMessageBox.critical(None, "Error", str(e))

    
    def capture_frame_and_stop_webcam(self):
        if hasattr(self, 'cap') and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                # Stop webcam feed
                self.cap.release()
                self.webcam_running = False
    
                # Perform face recognition on the captured frame
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
                # Load known encodings from the database
                known_face_encodings, known_face_names = self.load_encodings_from_database()
    
                # Process each detected face
                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                    name = "Unknown"
                    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
    
                    if matches[best_match_index]:
                        name = known_face_names[best_match_index]
    
                    # Draw a bounding box around the face
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
    
                    # Put the name below the bounding box
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left, bottom + 20), font, 1.0, (255, 255, 255), 1)
    
                # Convert the captured frame back to QPixmap for display
                height, width, channel = frame.shape
                bytes_per_line = 3 * width
                q_img = QtGui.QImage(frame.data, width, height, bytes_per_line, QtGui.QImage.Format_BGR888)
                self.webcam_image_viewer.setPixmap(QtGui.QPixmap.fromImage(q_img))
    

    def list_faces_in_dataset(self):
        try:
            db_connection = mysql.connector.connect(**mysql_config)
            cursor = db_connection.cursor()
            cursor.execute("SELECT image_name FROM images_store")
            names = [row[0] for row in cursor.fetchall()]
            cursor.close()
            db_connection.close()
            self.list_label.setText("\n".join(names) if names else "No faces found.")
        except Error as e:
            QMessageBox.critical(None, "Error", str(e))
# Main execution
# Main execution
if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    
    MainWindow.closeEvent = lambda event: app.quit()  
    
    MainWindow.show()
    app.exec_()
